In [2]:
import tensorflow as tf
import numpy as np

import warnings
## Tensorflow produces a lot of warnings. We generally want to suppress them. The below code does exactly that. 
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Using predefined estimators
* Instead of using TensorFlow basic API, it is often more convenient to use a higher-level API called **estimators**

* There are two ways to use the **estimators** API:
   * Pre-defined estimators.
   * Self-defined estimators.

* In this notebook we recreate the neural network defined first in notebook [3.Neural-Networks.ipynb](3.Neural-Networks.ipynb)  
using the predefined estimator `DNNClassifier`

## Benefits of Estimators

- Estimator-based models are independent of operating environment
    - local host
    - GPUs
    - CPU clusters

### More benefits
- Simplify model sharing between developers
- State of the art model architectures with more intuitive high-level code

Consult https://www.tensorflow.org/programmers_guide/estimators for more advantages of using Estimators as described by the developers of TensorFlow.

## Read Data
The MNist dataset is available from within TensorFlow tutorials.

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

# Wrap input as a function (THE "input function" will be defined below)
def input(dataset):
    return dataset.images, dataset.labels.astype(np.int32)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


##  Define feature columns

In [4]:
# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

## Define Neural Network
DNNClassifier is an estimator that implements a deep neural network classifier.

In [5]:
# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 256],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="./tmp/mnist_model_256_256"   # Location for storing checkpoints.
)

## Define training input function
* Supplies data for training, evaluation, prediction
* Should yield tuples of:
    - Python dict `features`: key = name of feature, value = array of feature values
    - Array `label` : label for every example

In [6]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

## Train the neural network
* Checkpoint used for "warm start"
* Checkpoints saved

In [11]:
tf.logging.set_verbosity(tf.logging.INFO)
classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10001 into ./tmp/mnist_model_256_256/model.ckpt.
INFO:tensorflow:loss = 0.8121629, step = 10001
INFO:tensorflow:global_step/sec: 291.444
INFO:tensorflow:loss = 0.9232541, step = 10101 (0.345 sec)
INFO:tensorflow:global_step/sec: 349.753
INFO:tensorflow:loss = 1.043865, step = 10201 (0.285 sec)
INFO:tensorflow:global_step/sec: 341.05
INFO:tensorflow:loss = 2.4103715, step = 10301 (0.293 sec)
INFO:tensorflow:global_step/sec: 335.136
INFO:tensorflow:loss = 2.737367, step = 10401 (0.300 sec)
INFO:tensorflow:global_step/sec: 290.02
INFO:tensorflow:loss = 3.1885724, step = 10501 (0.344 sec)
INFO:tensorflow:global_step/sec: 331.455
INFO:t

In [12]:
# Have a look at the checkpoint directory.
!ls -lrt ./tmp/mnist_model_256_256/ | tail

drwxr-xr-x 3 jovyan users      96 Jun  6 20:04 eval
-rw-r--r-- 1 jovyan users  138840 Jun  6 20:06 model.ckpt-10001.meta
-rw-r--r-- 1 jovyan users     808 Jun  6 20:06 model.ckpt-10001.index
-rw-r--r-- 1 jovyan users 3231880 Jun  6 20:06 model.ckpt-10001.data-00000-of-00001
-rw-r--r-- 1 jovyan users  313784 Jun  6 20:06 graph.pbtxt
-rw-r--r-- 1 jovyan users  138840 Jun  6 20:06 model.ckpt-11000.meta
-rw-r--r-- 1 jovyan users     808 Jun  6 20:06 model.ckpt-11000.index
-rw-r--r-- 1 jovyan users 3231880 Jun  6 20:06 model.ckpt-11000.data-00000-of-00001
-rw-r--r-- 1 jovyan users 1336467 Jun  6 20:06 events.out.tfevents.1528315423.aefc6083523a
-rw-r--r-- 1 jovyan users     226 Jun  6 20:06 checkpoint


## Define test input function

In [13]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False
)

## Evaluate accuracy

In [14]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"] 
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-20:06:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model_256_256/model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-20:06:56
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.976, average_loss = 0.07386734, global_step = 11000, loss = 9.350297

Test Accuracy: 97.600001%



## Summary
* We recreated the NN defined in [3.Neural-Networks.ipynb](3.Neural-Networks.ipynb) with a higher-level API.
* The **session** was replaced with calls to the estimator.
* Checkpoints are automatically available: allows continuation of training on a partially trained NN.
* **Next:** creating custom Estimators.